In [3]:
from langchain_core.documents import Document
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_groq import ChatGroq
from langchain_cohere import CohereEmbeddings, CohereRerank
from langchain_community.vectorstores import FAISS
import os

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())


True

#### Api key

In [14]:
groq_api_key = os.getenv("groq_api_key")
if groq_api_key is None:
    raise ValueError("Did not find groq_api_key, please add an environment variable `GROQ_API_KEY` which contains it, or pass `groq_api_key` as a named parameter.")

cohere_api_key = os.getenv("cohere_api_key")
if cohere_api_key is None:
    raise ValueError("Did not find cohere_api_key, please add an environment variable `COHERE_API_KEY`")

## LLM

In [11]:
# llm
llm = ChatGroq(
    model="llama3-8b-8192",
    api_key=groq_api_key,
    temperature=0,
    model_kwargs={
        "top_p": 0.95,
        "frequency_penalty": 0.1,
        "presence_penalty": 0.2,
    },
    verbose=True
)

# db
db = SQLDatabase.from_uri("sqlite:///../data/travel.sqlite")

## Create Document

In [12]:

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
context = toolkit.get_context()

In [18]:
print(context["table_info"])


CREATE TABLE aircrafts_data (
	aircraft_code TEXT, 
	model TEXT, 
	range INTEGER
)

/*
3 rows from aircrafts_data table:
aircraft_code	model	range
773	Boeing 777-300	11100
763	Boeing 767-300	7900
SU9	Sukhoi Superjet-100	3000
*/


CREATE TABLE airports_data (
	airport_code TEXT, 
	airport_name TEXT, 
	city TEXT, 
	coordinates TEXT, 
	timezone TEXT
)

/*
3 rows from airports_data table:
airport_code	airport_name	city	coordinates	timezone
ATL	Hartsfield-Jackson Atlanta International Airport	Atlanta	[33.6407, -84.4277]	America/New_York
PEK	Beijing Capital International Airport	Beijing	[40.0799, 116.6031]	Asia/Shanghai
DXB	Dubai International Airport	Dubai	[25.2532, 55.3657]	Asia/Dubai
*/


CREATE TABLE boarding_passes (
	ticket_no TEXT, 
	flight_id INTEGER, 
	boarding_no INTEGER, 
	seat_no TEXT
)

/*
3 rows from boarding_passes table:
ticket_no	flight_id	boarding_no	seat_no
0060005435212351	30625	1	2D
0060005435212386	30625	2	3G
0060005435212381	30625	3	4H
*/


CREATE TABLE bookings (
	bo

In [16]:
doc = Document(page_content=context["table_info"])

## Indexing VectoreStore